In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os.path
import nltk
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
print(stopwords.words('english'))
stop_words = set(stopwords.words('english'))
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
import string
import re
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
!pip install rouge
from rouge import Rouge
rouge=Rouge()

In [ ]:
df_clean = pd.read_csv('../input/hindi-text-short-and-large-summarization-corpus/test.csv',
                 lineterminator='\n')


In [ ]:
df_clean.head(50)
print(df_clean.shape)
del df_clean['headline']


In [ ]:
df_clean=df_clean.dropna()
df_clean=df_clean.reset_index(drop=True)

In [ ]:
df_clean.head(50)



In [ ]:
print(df_clean.shape[0])
#for i in range(0,len(documents)):


In [ ]:
def finding_all_unique_words_and_freq(words):
    words_unique = []
    word_freq = {}
    for word in words:
        if word not in words_unique:
            words_unique.append(word)
    for word in words_unique:
        word_freq[word] = words.count(word)
    return word_freq

def finding_freq_of_word_in_doc(word,words):
    freq = words.count(word)

In [ ]:
def finding_all_unique_words_and_freq_df(words):
    words_unique = []
    word_freq = {}
    for word in words:
        if word not in words_unique:
            words_unique.append(word)
    for word in words_unique:
        word_freq[word] = words.count(word)/len(words)
    return word_freq

In [ ]:

def genrate_tf_idf(probe_text):
  break_sentences=tokenizer.tokenize(probe_text)
  clean_sentences=[]
  for item in break_sentences:
    item=item.translate(str.maketrans('', '', string.punctuation))
    item=' '.join( [w for w in item.split() if len(w)>1] )
    clean_sentences.append(item)
    all_words = []
  dict_global = {}
  files_with_index = {}
  idx = 1
  for iter in clean_sentences:
    text = iter
    #text = re.sub(re.compile('\d'),'',text)
    words = word_tokenize(text)
    words = [word for word in words if len(words)>1]
    words = [word.lower() for word in words]
    words = [word for word in words if word not in stop_words]
    #words= [stemmer.stem(word) for word in words]
    dict_global.update(finding_all_unique_words_and_freq(words))
    #files_with_index[idx] = sentences
  unique_words_all = set(dict_global.keys())
  word_freq_in_doc={}
  id=0
  for iter in clean_sentences:
    text = iter
    #text = remove_special_characters(text)
    text = re.sub(re.compile('\d'),'',text)
    #sentences = sent_tokenize(text)
    words = word_tokenize(text)
    words = [word for word in words if len(words)>1]
    words = [word.lower() for word in words]
    words = [word for word in words if word not in stop_words]
    #words= [stemmer.stem(word) for word in words]
    word_freq_in_doc[id] = finding_all_unique_words_and_freq(words)
    id=id+1
  dic_array={}
  for words in  unique_words_all:
    dic_array[words]=[]
  #print(dic_array)
  #print('')
  idx=1
  word_freq_in_sent={}
  for iter in clean_sentences:
    text = iter
    #text = remove_special_characters(text)
    #text = re.sub(re.compile('\d'),'',text)
    #sentences = sent_tokenize(text)
    words = word_tokenize(text)
    words = [word for word in words if len(words)>1]
    words = [word.lower() for word in words]
    words = [word for word in words if word not in stop_words]
    #words= [stemmer.stem(word) for word in words]
    word_freq_in_doc = finding_all_unique_words_and_freq(words)
    word_freq_in_sent[idx]=word_freq_in_doc
    #print(word_freq_in_doc)
    for word in word_freq_in_doc.keys():
      word_freq_doc=[idx, word_freq_in_doc[word]]
      if word in dic_array.keys():
        dic_array[word].append(word_freq_doc)
    idx=idx+1
  tf_docs={}
  idx=1
  for iter in clean_sentences:
    text = iter
    #text = remove_special_characters(text)
    #text = re.sub(re.compile('\d'),'',text)
    #sentences = sent_tokenize(text)
    words = word_tokenize(text)
    words = [word for word in words if len(words)>1]
    words = [word.lower() for word in words]
    words = [word for word in words if word not in stop_words]
    #words= [stemmer.stem(word) for word in words]
    word_freq_in_doc = finding_all_unique_words_and_freq_df(words)
    tf_docs[idx]=word_freq_in_doc
    idx=idx+1
  idf={}
  num_docs=len(clean_sentences)
  for keys in dic_array:
    num_doc_orrurance=dic_array[keys]
    idf_val=num_docs/(len(num_doc_orrurance)+1)
    idf_val=math.log(idf_val) # add log here 
    if(idf_val < 0):
      idf_val=0
    idf[keys]=idf_val
  tf_idf={}
  idx=1
  for iter in clean_sentences:
    doc_idx=idx
    df_values=tf_docs[doc_idx]
    idf_doc={}
    for keys in df_values:
      val=df_values[keys]*idf[keys]
      idf_doc[keys]=val
    tf_idf[doc_idx]=idf_doc
    idx=idx+1
  unique_word_list=list(unique_words_all)
  return tf_idf,unique_word_list


In [ ]:
except_i=[8,]

First 0.7 sigma

In [ ]:
rouge_1_list_p=[]
rouge_1_list_r=[]
rouge_1_list_f=[]
rouge_2_list_p=[]
rouge_2_list_r=[]
rouge_2_list_f=[]
rouge_l_list_p=[]
rouge_l_list_r=[]
rouge_l_list_f=[]
for i in range(0,df_clean.shape[0]):
    print(i)
    probe_text=df_clean.iloc[i][1]
    ground_truth=df_clean.iloc[i][0]
    #print(ground_truth)
    tf_idf,unique_word_list=genrate_tf_idf(probe_text)
    break_sentences=tokenizer.tokenize(probe_text)
    clean_sentences=[]
    stop_words = set(stopwords.words('english'))
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    clean_sentences=[]
    for item in break_sentences:
        item=item.translate(str.maketrans('', '', string.punctuation))
        item=' '.join( [w for w in item.split() if len(w)>1] )
        clean_sentences.append(item)
    #print(len(clean_sentences))
    #print(tf_idf)
    #print(unique_word_list)
    svd_matrix=np.zeros([len(unique_word_list),len(clean_sentences)])
    for i in range(1,len(clean_sentences)+1):
        doc_item=tf_idf[i]
        for val in doc_item:
          index=unique_word_list.index(val)
          svd_matrix[index,i-1]=doc_item[val]
    u,sigma,v_trans=np.linalg.svd(svd_matrix,full_matrices=False,compute_uv=True)
    sum_sigma=np.sum(sigma)
    iter_sum=0
    check_coverage=0
    for i in range(0,len(sigma)):
        iter_sum=iter_sum+sigma[i]
        percent_contribution=iter_sum/sum_sigma
        #print("the percent contributuin is ",percent_contribution)
        if(percent_contribution<0.70):
          check_coverage=i+1
    chosen_sen_index=[]
    for i in range(0,check_coverage):
        sel_v_val=v_trans[:,i]
        #print(sel_v_val)
        sel_sen=sel_v_val.argmax(axis=0)
        #print(sel_sen)
        chosen_sen_index.append(sel_sen)
    summ_list=[]
    #print(chosen_sen_index)
    for i in chosen_sen_index:
        #print(clean_sentences[i])
        summ_list.append(clean_sentences[i])
    summ = '.'.join(summ_list)
    #print("The summary is",summ)
    #clean_summ_join=''
    #clean_summ_join=clean_summ_join.join(clean_summ[i]) 
    if(len(summ) > 0 and len(ground_truth)>0):
        rge=rouge.get_scores(summ,ground_truth)[0]
        #print(rge)
        tmp_rge1=(rge['rouge-1'])
        tmp_rge2=(rge['rouge-2'])
        tmp_rgel=(rge['rouge-l'])
        rouge_1_list_p.append(tmp_rge1['p'])
        rouge_1_list_r.append(tmp_rge1['r'])
        rouge_1_list_f.append(tmp_rge1['f'])
        rouge_2_list_p.append(tmp_rge2['p'])
        rouge_2_list_r.append(tmp_rge2['r'])
        rouge_2_list_f.append(tmp_rge2['f'])
        rouge_l_list_p.append(tmp_rgel['p'])
        rouge_l_list_r.append(tmp_rgel['r'])
        rouge_l_list_f.append(tmp_rgel['f'])

    




    
    


In [ ]:
rouge_1_list_p_score=np.mean(np.array(rouge_1_list_p))
print("rouge_1_list_p_score",rouge_1_list_p_score)
rouge_1_list_r_score=np.mean(np.array(rouge_1_list_r))
print("rouge_1_list_r_score",rouge_1_list_r_score)
rouge_1_list_f_scopre=np.mean(np.array(rouge_1_list_f))
print("rouge_1_list_f_scopre",rouge_1_list_f_scopre)
rouge_2_list_p_score=np.mean(np.array(rouge_2_list_p))
print("rouge_2_list_p_score",rouge_2_list_p_score)
rouge_2_list_r_score=np.mean(np.array(rouge_2_list_r))
print("rouge_2_list_r_score",rouge_2_list_r_score)
rouge_2_list_f_score=np.mean(np.array(rouge_2_list_f))
print("rouge_2_list_f_score",rouge_2_list_f_score)
rouge_l_list_p_score=np.mean(np.array(rouge_l_list_p))
print("rouge_l_list_p_score",rouge_l_list_p_score)
rouge_l_list_r_score=np.mean(np.array(rouge_l_list_r))
print("rouge_l_list_r_score",rouge_l_list_r_score)
rouge_l_list_f_score=np.mean(np.array(rouge_l_list_f))
print("rouge_l_list_f_score",rouge_l_list_f_score)

In [ ]:
exc_i=[111,277,295,342,391,399,887,950,1018,1048,1204,1300,1354,1468]

In [ ]:
rouge_1_list_p=[]
rouge_1_list_r=[]
rouge_1_list_f=[]
rouge_2_list_p=[]
rouge_2_list_r=[]
rouge_2_list_f=[]
rouge_l_list_p=[]
rouge_l_list_r=[]
rouge_l_list_f=[]
for i in range(0,df_clean.shape[0]):
    #print(i)
    probe_text=df_clean.iloc[i][1]
    ground_truth=df_clean.iloc[i][0]
    #print(ground_truth)
    tf_idf,unique_word_list=genrate_tf_idf(probe_text)
    break_sentences=tokenizer.tokenize(probe_text)
    clean_sentences=[]
    stop_words = set(stopwords.words('english'))
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    clean_sentences=[]
    for item in break_sentences:
        item=item.translate(str.maketrans('', '', string.punctuation))
        item=' '.join( [w for w in item.split() if len(w)>1] )
        clean_sentences.append(item)
    svd_matrix=np.zeros([len(unique_word_list),len(clean_sentences)])
    for i in range(1,len(clean_sentences)+1):
        doc_item=tf_idf[i]
        for val in doc_item:
          index=unique_word_list.index(val)
          svd_matrix[index,i-1]=doc_item[val]
    u,sigma,v_trans=np.linalg.svd(svd_matrix,full_matrices=False,compute_uv=True)
    sum_sigma=np.sum(sigma)
    iter_sum=0
    check_coverage=0 
    u,sigma,v_trans=np.linalg.svd(svd_matrix,full_matrices=False,compute_uv=True)
    sum_sigma=np.sum(sigma)
    iter_sum=0
    check_coverage=0
    sen_score_sigma=np.zeros(len(clean_sentences))
    for i in range(0,len(clean_sentences)):
        tmp=(v_trans[:,i]**2) * (sigma**2)
        tmp_sqrt=np.sqrt(np.sum(tmp))
        sen_score_sigma[i]=tmp_sqrt
    sum_score=np.sum(sen_score_sigma)
    sorted_array=np.sort(sen_score_sigma)[::-1]
    chosen_sen_index=[]
    sum=0
    iter_val=1
    for i in range(0,len(clean_sentences)):
        sum=sum+sorted_array[i]
        percentage=sum/sum_score
        iter_val=iter_val+1
        if(percentage > 0.5):
          break
    top_n=sen_score_sigma.argsort()[-iter_val:][::-1]
    summ_list=[]
    for i in top_n:
    #print(clean_sentences[i]) 
        summ_list.append(clean_sentences[i])
    summ=''
    summ = '.'.join(summ_list)
    #print(summ)
    if(len(summ) > 0 and len(ground_truth)>0):
        try:
            rge=rouge.get_scores(summ,ground_truth)[0]
        except RecursionError :
            print("recursive error")

        #print(rge)
    tmp_rge1=(rge['rouge-1'])
    tmp_rge2=(rge['rouge-2'])
    tmp_rgel=(rge['rouge-l'])
    rouge_1_list_p.append(tmp_rge1['p'])
    rouge_1_list_r.append(tmp_rge1['r'])
    rouge_1_list_f.append(tmp_rge1['f'])
    rouge_2_list_p.append(tmp_rge2['p'])
    rouge_2_list_r.append(tmp_rge2['r'])
    rouge_2_list_f.append(tmp_rge2['f'])
    rouge_l_list_p.append(tmp_rgel['p'])
    rouge_l_list_r.append(tmp_rgel['r'])
    rouge_l_list_f.append(tmp_rgel['f'])
   
    




    
    


In [ ]:
rouge_1_list_p_score=np.mean(np.array(rouge_1_list_p))
print("rouge_1_list_p_score",rouge_1_list_p_score)
rouge_1_list_r_score=np.mean(np.array(rouge_1_list_r))
print("rouge_1_list_r_score",rouge_1_list_r_score)
rouge_1_list_f_scopre=np.mean(np.array(rouge_1_list_f))
print("rouge_1_list_f_scopre",rouge_1_list_f_scopre)
rouge_2_list_p_score=np.mean(np.array(rouge_2_list_p))
print("rouge_2_list_p_score",rouge_2_list_p_score)
rouge_2_list_r_score=np.mean(np.array(rouge_2_list_r))
print("rouge_2_list_r_score",rouge_2_list_r_score)
rouge_2_list_f_score=np.mean(np.array(rouge_2_list_f))
print("rouge_2_list_f_score",rouge_2_list_f_score)
rouge_l_list_p_score=np.mean(np.array(rouge_l_list_p))
print("rouge_l_list_p_score",rouge_l_list_p_score)
rouge_l_list_r_score=np.mean(np.array(rouge_l_list_r))
print("rouge_l_list_r_score",rouge_l_list_r_score)
rouge_l_list_f_score=np.mean(np.array(rouge_l_list_f))
print("rouge_l_list_f_score",rouge_l_list_f_score)

In [ ]:
except_i=[474,1967,2393,4503,7204,7542,7686,9409,12604,15729]

In [ ]:
rouge_1_list_p=[]
rouge_1_list_r=[]
rouge_1_list_f=[]
rouge_2_list_p=[]
rouge_2_list_r=[]
rouge_2_list_f=[]
rouge_l_list_p=[]
rouge_l_list_r=[]
rouge_l_list_f=[]
for i in range(0,df_clean.shape[0]):
    #print(i)
    if(i % 1000 == 0):
        print(i)
    if i not in except_i:
        probe_text=df_clean.iloc[i][1]
        ground_truth=df_clean.iloc[i][0]
        #print(ground_truth)
        break_sentences=tokenizer.tokenize(probe_text)
        clean_sentences=[]
        stop_words = set(stopwords.words('english'))
        punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
        clean_sentences=[]
        for item in break_sentences:
            item=item.translate(str.maketrans('', '', string.punctuation))
            item=' '.join( [w for w in item.split() if len(w)>1] )
            clean_sentences.append(item)
        clean_sentences.append(item)
        cln_summ=''
        cln_summ = '.'.join(clean_sentences)
        tfidf = CountVectorizer()
        response = tfidf.fit_transform([cln_summ])
        #print(response)
        unique_word_list = tfidf.get_feature_names_out()
        tf_idf={}
        for col in response.nonzero()[1]:
            tf_idf[unique_word_list[col]]= response[0, col]
        svd_matrix=np.zeros([len(unique_word_list),len(clean_sentences)])
        for i in range(0,len(clean_sentences)):
            sen=clean_sentences[i].split()
            for val in sen:
                index=np.where(unique_word_list == val)
                index_length=len(index[0])
                if(index_length == 1):
                    svd_matrix[index,i]=tf_idf[val]
      #print(svd_matrix)
        u,sigma,v_trans=np.linalg.svd(svd_matrix,full_matrices=False,compute_uv=True)
        sum_sigma=np.sum(sigma)
        iter_sum=0
        check_coverage=0
        for i in range(0,len(sigma)):
            iter_sum=iter_sum+sigma[i]
            percent_contribution=iter_sum/sum_sigma
            if(percent_contribution<0.70):
                check_coverage=i+1
        chosen_sen_index=[]
        for i in range(0,check_coverage):
            sel_v_val=v_trans[:,i]
            sel_sen=sel_v_val.argmax(axis=0)
            chosen_sen_index.append(sel_sen)
        summ_list=[]
        for i in chosen_sen_index:
        #print(clean_sentences[i])
            summ_list.append(clean_sentences[i])
        summ = '.'.join(summ_list)
      #print(summ)
      #print(clean_summ_join)
        if(len(summ) > 0 and len(ground_truth)>0):
            try:
                rge=rouge.get_scores(summ,ground_truth)[0]
            except RecursionError :
                print("recursive error")
            #print(rge)
        tmp_rge1=(rge['rouge-1'])
        tmp_rge2=(rge['rouge-2'])
        tmp_rgel=(rge['rouge-l'])
        rouge_1_list_p.append(tmp_rge1['p'])
        rouge_1_list_r.append(tmp_rge1['r'])
        rouge_1_list_f.append(tmp_rge1['f'])
        rouge_2_list_p.append(tmp_rge2['p'])
        rouge_2_list_r.append(tmp_rge2['r'])
        rouge_2_list_f.append(tmp_rge2['f'])
        rouge_l_list_p.append(tmp_rgel['p'])
        rouge_l_list_r.append(tmp_rgel['r'])
        rouge_l_list_f.append(tmp_rgel['f'])
    




    
    


In [ ]:
rouge_1_list_p_score=np.mean(np.array(rouge_1_list_p))
print("rouge_1_list_p_score",rouge_1_list_p_score)
rouge_1_list_r_score=np.mean(np.array(rouge_1_list_r))
print("rouge_1_list_r_score",rouge_1_list_r_score)
rouge_1_list_f_scopre=np.mean(np.array(rouge_1_list_f))
print("rouge_1_list_f_scopre",rouge_1_list_f_scopre)
rouge_2_list_p_score=np.mean(np.array(rouge_2_list_p))
print("rouge_2_list_p_score",rouge_2_list_p_score)
rouge_2_list_r_score=np.mean(np.array(rouge_2_list_r))
print("rouge_2_list_r_score",rouge_2_list_r_score)
rouge_2_list_f_score=np.mean(np.array(rouge_2_list_f))
print("rouge_2_list_f_score",rouge_2_list_f_score)
rouge_l_list_p_score=np.mean(np.array(rouge_l_list_p))
print("rouge_l_list_p_score",rouge_l_list_p_score)
rouge_l_list_r_score=np.mean(np.array(rouge_l_list_r))
print("rouge_l_list_r_score",rouge_l_list_r_score)
rouge_l_list_f_score=np.mean(np.array(rouge_l_list_f))
print("rouge_l_list_f_score",rouge_l_list_f_score)

In [ ]:
rouge_1_list_p=[]
rouge_1_list_r=[]
rouge_1_list_f=[]
rouge_2_list_p=[]
rouge_2_list_r=[]
rouge_2_list_f=[]
rouge_l_list_p=[]
rouge_l_list_r=[]
rouge_l_list_f=[]
for i in range(0,df_clean.shape[0]):
    if(i % 1000 ==0):
        print(i)
    for i not in except_i:
        probe_text=df_clean.iloc[i][1]
        ground_truth=df_clean.iloc[i][0]
        #print(ground_truth)
        break_sentences=tokenizer.tokenize(probe_text)
        clean_sentences=[]
        stop_words = set(stopwords.words('english'))
        punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
        clean_sentences=[]
        for item in break_sentences:
            item=item.translate(str.maketrans('', '', string.punctuation))
            item=' '.join( [w for w in item.split() if len(w)>1] )
            clean_sentences.append(item)
        clean_sentences.append(item)
        cln_summ=''
        cln_summ = '.'.join(clean_sentences)
        tfidf = CountVectorizer()
        response = tfidf.fit_transform([cln_summ])
        #print(response)
        unique_word_list = tfidf.get_feature_names_out()
        tf_idf={}
        for col in response.nonzero()[1]:
            tf_idf[unique_word_list[col]]= response[0, col]
        svd_matrix=np.zeros([len(unique_word_list),len(clean_sentences)])
        for i in range(0,len(clean_sentences)):
            sen=clean_sentences[i].split()
            for val in sen:
                index=np.where(unique_word_list == val)
                index_length=len(index[0])
                if(index_length == 1):
                    svd_matrix[index,i]=tf_idf[val]
      #print(svd_matrix)
        u,sigma,v_trans=np.linalg.svd(svd_matrix,full_matrices=False,compute_uv=True)
        sum_sigma=np.sum(sigma)
        iter_sum=0
        check_coverage=0
        for i in range(0,len(sigma)):
            iter_sum=iter_sum+sigma[i]
            percent_contribution=iter_sum/sum_sigma
            if(percent_contribution<0.70):
                check_coverage=i+1
        chosen_sen_index=[]
        for i in range(0,check_coverage):
            sel_v_val=v_trans[:,i]
            sel_sen=sel_v_val.argmax(axis=0)
            chosen_sen_index.append(sel_sen)
        summ_list=[]
        for i in chosen_sen_index:
        #print(clean_sentences[i])
            summ_list.append(clean_sentences[i])
        summ = '.'.join(summ_list)
      #print(summ)
      #print(clean_summ_join)
        if(len(summ) > 0 and len(ground_truth)>0):
            rge=rouge.get_scores(summ,ground_truth)[0]
            print(rge)
        tmp_rge1=(rge['rouge-1'])
        tmp_rge2=(rge['rouge-2'])
        tmp_rgel=(rge['rouge-l'])
        rouge_1_list_p.append(tmp_rge1['p'])
        rouge_1_list_r.append(tmp_rge1['r'])
        rouge_1_list_f.append(tmp_rge1['f'])
        rouge_2_list_p.append(tmp_rge2['p'])
        rouge_2_list_r.append(tmp_rge2['r'])
        rouge_2_list_f.append(tmp_rge2['f'])
        rouge_l_list_p.append(tmp_rgel['p'])
        rouge_l_list_r.append(tmp_rgel['r'])
        rouge_l_list_f.append(tmp_rgel['f'])


In [ ]:
rouge_1_list_p_score=np.mean(np.array(rouge_1_list_p))
print("rouge_1_list_p_score",rouge_1_list_p_score)
rouge_1_list_r_score=np.mean(np.array(rouge_1_list_r))
print("rouge_1_list_r_score",rouge_1_list_r_score)
rouge_1_list_f_scopre=np.mean(np.array(rouge_1_list_f))
print("rouge_1_list_f_scopre",rouge_1_list_f_scopre)
rouge_2_list_p_score=np.mean(np.array(rouge_2_list_p))
print("rouge_2_list_p_score",rouge_2_list_p_score)
rouge_2_list_r_score=np.mean(np.array(rouge_2_list_r))
print("rouge_2_list_r_score",rouge_2_list_r_score)
rouge_2_list_f_score=np.mean(np.array(rouge_2_list_f))
print("rouge_2_list_f_score",rouge_2_list_f_score)
rouge_l_list_p_score=np.mean(np.array(rouge_l_list_p))
print("rouge_l_list_p_score",rouge_l_list_p_score)
rouge_l_list_r_score=np.mean(np.array(rouge_l_list_r))
print("rouge_l_list_r_score",rouge_l_list_r_score)
rouge_l_list_f_score=np.mean(np.array(rouge_l_list_f))
print("rouge_l_list_f_score",rouge_l_list_f_score)

My imp 2 clear sentence

In [ ]:
rouge_1_list_p=[]
rouge_1_list_r=[]
rouge_1_list_f=[]
rouge_2_list_p=[]
rouge_2_list_r=[]
rouge_2_list_f=[]
rouge_l_list_p=[]
rouge_l_list_r=[]
rouge_l_list_f=[]
for i in range(0,df_clean.shape[0]):
    #print(i)
    if(i % 1000 ==0):
        print(i)
    if i not in except_i:
        probe_text=df_clean.iloc[i][1]
        ground_truth=df_clean.iloc[i][0]
        #print(ground_truth)
        break_sentences=tokenizer.tokenize(probe_text)
        clean_sentences=[]
        stop_words = set(stopwords.words('english'))
        punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
        clean_sentences=[]
        for item in break_sentences:
            item=item.translate(str.maketrans('', '', string.punctuation))
            item=' '.join( [w for w in item.split() if len(w)>1] )
            clean_sentences.append(item)
        clean_sentences.append(item)
        cln_summ=''
        cln_summ = '.'.join(clean_sentences)
        tfidf = CountVectorizer()
        response = tfidf.fit_transform([cln_summ])
        #print(response)
        unique_word_list = tfidf.get_feature_names_out()
        tf_idf={}
        for col in response.nonzero()[1]:
            tf_idf[unique_word_list[col]]= response[0, col]
        svd_matrix=np.zeros([len(unique_word_list),len(clean_sentences)])
        for i in range(0,len(clean_sentences)):
            sen=clean_sentences[i].split()
            for val in sen:
                index=np.where(unique_word_list == val)
                index_length=len(index[0])
                if(index_length == 1):
                    svd_matrix[index,i]=tf_idf[val]
        u,sigma,v_trans=np.linalg.svd(svd_matrix,full_matrices=False,compute_uv=True)
        sum_sigma=np.sum(sigma)
        iter_sum=0
        check_coverage=0 
        u,sigma,v_trans=np.linalg.svd(svd_matrix,full_matrices=False,compute_uv=True)
        sum_sigma=np.sum(sigma)
        iter_sum=0
        check_coverage=0
        sen_score_sigma=np.zeros(len(clean_sentences))
        for i in range(0,len(clean_sentences)):
            tmp=(v_trans[:,i]**2) * (sigma**2)
            tmp_sqrt=np.sqrt(np.sum(tmp))
            sen_score_sigma[i]=tmp_sqrt
        sum_score=np.sum(sen_score_sigma)
        sorted_array=np.sort(sen_score_sigma)[::-1]
        chosen_sen_index=[]
        sum=0
        iter_val=1
        for i in range(0,len(clean_sentences)):
            sum=sum+sorted_array[i]
            percentage=sum/sum_score
            iter_val=iter_val+1
            if(percentage > 0.5):
              break
        top_n=sen_score_sigma.argsort()[-iter_val:][::-1]
        summ_list=[]
        for i in top_n:
        #print(clean_sentences[i])
            summ_list.append(clean_sentences[i])
        summ=''
        summ = '.'.join(summ_list)
        #print(summ)
        if(len(summ) > 0 and len(ground_truth)>0):
            try:
                rge=rouge.get_scores(summ,ground_truth)[0]
            except RecursionError :
                print("recursive error")
        tmp_rge1=(rge['rouge-1'])
        tmp_rge2=(rge['rouge-2'])
        tmp_rgel=(rge['rouge-l'])
        rouge_1_list_p.append(tmp_rge1['p'])
        rouge_1_list_r.append(tmp_rge1['r'])
        rouge_1_list_f.append(tmp_rge1['f'])
        rouge_2_list_p.append(tmp_rge2['p'])
        rouge_2_list_r.append(tmp_rge2['r'])
        rouge_2_list_f.append(tmp_rge2['f'])
        rouge_l_list_p.append(tmp_rgel['p'])
        rouge_l_list_r.append(tmp_rgel['r'])
        rouge_l_list_f.append(tmp_rgel['f'])


In [ ]:
rouge_1_list_p_score=np.mean(np.array(rouge_1_list_p))
print("rouge_1_list_p_score",rouge_1_list_p_score)
rouge_1_list_r_score=np.mean(np.array(rouge_1_list_r))
print("rouge_1_list_r_score",rouge_1_list_r_score)
rouge_1_list_f_scopre=np.mean(np.array(rouge_1_list_f))
print("rouge_1_list_f_scopre",rouge_1_list_f_scopre)
rouge_2_list_p_score=np.mean(np.array(rouge_2_list_p))
print("rouge_2_list_p_score",rouge_2_list_p_score)
rouge_2_list_r_score=np.mean(np.array(rouge_2_list_r))
print("rouge_2_list_r_score",rouge_2_list_r_score)
rouge_2_list_f_score=np.mean(np.array(rouge_2_list_f))
print("rouge_2_list_f_score",rouge_2_list_f_score)
rouge_l_list_p_score=np.mean(np.array(rouge_l_list_p))
print("rouge_l_list_p_score",rouge_l_list_p_score)
rouge_l_list_r_score=np.mean(np.array(rouge_l_list_r))
print("rouge_l_list_r_score",rouge_l_list_r_score)
rouge_l_list_f_score=np.mean(np.array(rouge_l_list_f))
print("rouge_l_list_f_score",rouge_l_list_f_score)